# Análisis Exploratorio de Datos

## Carga de paquetes 

In [ ]:
import pandas as pd
import numpy  as np
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import streamlit as st
import plotly.express as px
from datetime import datetime

# Se hace la conexión local de la BBDD
conn = sqlite3.connect(r"database.sqlite")

cur = conn.cursor()

In [2]:
consulta_ntablas = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"

print("Entidades en la base de datos:")
entidades = pd.read_sql_query(sql = consulta_ntablas, con = conn)
entidades

Entidades en la base de datos:


,name
0,Atributos_Jugador
1,Country
2,Equipo
3,Jugadores
4,League
5,Liga
6,Match
7,Partido
8,Player
9,Player_Attributes


## Denormalización de la BBDD

En este paso se aplicó el enfoque de denormalización propuesto por Ralph Kimball, con el propósito de optimizar el acceso y la velocidad de consulta de los datos. A diferencia de los modelos altamente normalizados, donde la información se distribuye en múltiples tablas para evitar redundancias, la denormalización busca crear estructuras más simples y directas que faciliten el análisis y reduzcan la necesidad de realizar uniones (joins) complejas. Este enfoque resulta especialmente útil en contextos de análisis de datos o inteligencia de negocio, donde la eficiencia en las consultas y la claridad en la estructura de la base de datos son prioritarias.

En el código desarrollado, se transformó las principales tablas de la base de datos original en versiones más accesibles y consolidadas. Por ejemplo, se creó la tabla Liga a partir de la combinación de las tablas League y Country, integrando el nombre del país directamente en la estructura de la liga. Además, se generaron las tablas Jugadores, Atributos_Jugador, Partido y Equipo, seleccionando únicamente los campos más relevantes y aplicando transformaciones para estandarizar los tipos de datos, como la conversión de fechas. De esta manera, se obtuvo un modelo más ágil, con menor complejidad relacional y preparado para realizar análisis de manera más eficiente.


In [ ]:
jugadores = pd.read_sql("SELECT player_api_id, player_name, birthday, height, weight FROM Player", conn)


cur.execute("""
CREATE TABLE IF NOT EXISTS Jugadores AS
SELECT player_api_id, player_name, birthday, height, weight FROM Player;
""")
cur.execute("UPDATE Jugadores SET birthday = DATE(birthday);")
conn.commit()
jugadores = pd.read_sql("SELECT * FROM Jugadores", conn)

cur.execute("""
CREATE TABLE IF NOT EXISTS Liga AS
SELECT
    T2.id AS id_country_league,
    T2.name AS name_country,
    T1.name AS name_league
FROM
    League AS T1
INNER JOIN
    Country AS T2 ON T1.country_id = T2.id;
""")
conn.commit()
Liga = pd.read_sql("SELECT * FROM Liga", conn)

cur.execute("""
CREATE TABLE IF NOT EXISTS Partido AS
SELECT match_api_id, league_id, season, date, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal, home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7, home_player_8, home_player_9, home_player_10, home_player_11, away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7, away_player_8, away_player_9, away_player_10, away_player_11
FROM Match;
""")
cur.execute("UPDATE Partido SET date = DATE(date);")
conn.commit()
Partido = pd.read_sql("SELECT * FROM Partido", conn)

cur.execute("""
CREATE TABLE IF NOT EXISTS Atributos_Jugador AS
SELECT
    player_api_id,
    date,
    overall_rating,
    potential,
    preferred_foot,
    acceleration,
    sprint_speed,
    dribbling,
    agility,
    ball_control,
    long_shots,
    shot_power,
    penalties,
    volleys,
    gk_diving,
    gk_handling,
    gk_kicking,
    gk_positioning,
    gk_reflexes
FROM
    Player_Attributes;
""")
cur.execute("UPDATE Atributos_Jugador SET date = DATE(date);")
conn.commit()
Atributos_Jugadores = pd.read_sql("SELECT * FROM Atributos_Jugador", conn)

cur.execute("""
CREATE TABLE IF NOT EXISTS Equipo AS
SELECT team_api_id, team_short_name, team_long_name
FROM Team;
""")
conn.commit()
Equipo = pd.read_sql("SELECT * FROM Equipo", conn)

conn.close()

In [4]:
def renombrar_columnas(df, mapeo):
    return df.rename(columns=mapeo)

mapeo_jugadores = {
    'player_api_id': 'ID_Jugador', 
    'player_name': 'Nombre_Jugador', 
    'birthday': 'Fecha_Nacimiento', 
    'height': 'Altura', 
    'weight': 'Peso'
}
mapeo_atributos = {
    'player_api_id': 'ID_Jugador', 
    'date': "Fecha_Temporada", 
    'overall_rating': "Calificación_General", 
    'potential': 'Potencial', 
    'preferred_foot': 'Pie_Preferido',
    'acceleration': "Aceleración", 
    'sprint_speed': "Velocidad_Sprint", 
    'dribbling': 'Regate', 
    'agility': 'Agilidad', 
    'ball_control': "Control_Balón",
    'long_shots': 'Tiros_Largos', 
    'shot_power': 'Potencia_Tiro', 
    'penalties': 'Penaltis', 
    'volleys': 'Voleas',
    'gk_diving': 'Portero_Salto', 
    'gk_handling': 'Portero_Manejo', 
    'gk_kicking': 'Portero_Patada', 
    'gk_positioning': 'Portero_Posicionamiento', 
    'gk_reflexes': 'Portero_Reflejos'
}
mapeo_equipo = {
    'team_api_id': 'ID_Equipo', 
    'team_short_name': 'Nombre_Corto_Equipo', 
    'team_long_name': 'Nombre_Largo_Equipo'
}
mapeo_liga = {
    'id_country_league': 'ID_Pais_Liga', 
    'name_country': 'Nombre_Pais', 
    'name_league': 'Nombre_Liga'
}
mapeo_partido = {
    'match_api_id': 'ID_Partido', 
    'league_id': 'ID_Pais_Liga',
    'season': 'Temporada',
    'date': 'Fecha_Partido',
    'home_team_api_id': 'ID_Equipo_Local',
    'away_team_api_id': 'ID_Equipo_Visitante',
    'home_team_goal': 'Goles_Local',
    'away_team_goal': 'Goles_Visitante',
    'home_player_1' : 'Jugador_Local_1',
    'home_player_2' : 'Jugador_Local_2',
    'home_player_3' : 'Jugador_Local_3',
    'home_player_4' : 'Jugador_Local_4',
    'home_player_5' : 'Jugador_Local_5',
    'home_player_6' : 'Jugador_Local_6',
    'home_player_7' : 'Jugador_Local_7',
    'home_player_8' : 'Jugador_Local_8',
    'home_player_9' : 'Jugador_Local_9',
    'home_player_10' : 'Jugador_Local_10',
    'home_player_11' : 'Jugador_Local_11',
    'away_player_1' : 'Jugador_Visitante_1',
    'away_player_2' : 'Jugador_Visitante_2',
    'away_player_3' : 'Jugador_Visitante_3',
    'away_player_4' : 'Jugador_Visitante_4',
    'away_player_5' : 'Jugador_Visitante_5',
    'away_player_6' : 'Jugador_Visitante_6',
    'away_player_7' : 'Jugador_Visitante_7',
    'away_player_8' : 'Jugador_Visitante_8',
    'away_player_9' : 'Jugador_Visitante_9',
    'away_player_10' : 'Jugador_Visitante_10',
    'away_player_11' : 'Jugador_Visitante_11'
}

Jugadores = renombrar_columnas(Jugadores, mapeo_jugadores)
Atributos_Jugadores = renombrar_columnas(Atributos_Jugadores, mapeo_atributos)
Equipo = renombrar_columnas(Equipo, mapeo_equipo)
Liga = renombrar_columnas(Liga, mapeo_liga)
Partido = renombrar_columnas(Partido, mapeo_partido)

Partido = Partido.dropna(how='any')


In [5]:
Jugadores['Peso'] = pd.to_numeric(Jugadores['Peso'], errors='coerce')
Jugadores['Peso'] = (Jugadores['Peso'] * 0.453592).round(2)

Atributos_Jugadores['Fecha_Temporada'] = pd.to_datetime(Atributos_Jugadores['Fecha_Temporada'], errors='coerce')
Atributos_Jugadores['Fecha_Temporada'] = Atributos_Jugadores['Fecha_Temporada'].dt.strftime('%d-%m-%y')

Atributos_Jugadores['Pie_Preferido'] = Atributos_Jugadores['Pie_Preferido'].replace({
    'right': 'derecho',
    'left': 'zurdo'
})


Partido['Fecha_Partido'] = pd.to_datetime(Partido['Fecha_Partido'], errors='coerce')
Partido['Fecha_Partido'] = Partido['Fecha_Partido'].dt.strftime('%d-%m-%y')

traduccion_paises = {
    'Belgium': "Bélgica",
    'England': 'Inglaterra',
    'France': 'Francia',
    'Germany': 'Alemania',
    'Italy': 'Italia',
    'Netherlands': "Países Bajos",
    'Poland': 'Polonia',
    'Portugal': 'Portugal',
    'Scotland': 'Escocia',
    'Spain': "España",
    'Switzerland': 'Suiza',
}

traduccion_ligas = {
    'Belgium Jupiler League': 'Jupiler Pro League',
    'England Premier League': 'Premier League',
    'France Ligue 1': 'Ligue 1',
    'Germany 1. Bundesliga': 'Bundesliga',
    'Italy Serie A': 'Serie A',
    'Netherlands Eredivisie': 'Eredivisie',
    'Poland Ekstraklasa': 'Ekstraklasa',
    'Portugal Liga ZON Sagres': 'Primeira Liga',
    'Scotland Premier League': 'Scottish Premiership',
    'Spain LIGA BBVA': 'Liga BBVA',
    'Switzerland Super League': 'Super League',
}

Liga['Nombre_Pais'] = Liga['Nombre_Pais'].replace(traduccion_paises)
Liga['Nombre_Liga'] = Liga['Nombre_Liga'].replace(traduccion_ligas)


In [6]:
Partido_Liga_Temp = pd.merge(
    Partido,
    Equipo[['ID_Equipo', 'Nombre_Largo_Equipo']],
    left_on='ID_Equipo_Local',
    right_on='ID_Equipo',
    how='left',
    suffixes=('', '_local')
)
Partido_Liga_Temp = Partido_Liga_Temp.rename(columns={'Nombre_Largo_Equipo': 'Nombre_Equipo_Local'}).drop(columns=['ID_Equipo'])

Partido_Liga_Temp = pd.merge(
    Partido_Liga_Temp,
    Equipo[['ID_Equipo', 'Nombre_Largo_Equipo']],
    left_on='ID_Equipo_Visitante',
    right_on='ID_Equipo',
    how='left',
    suffixes=('', '_visitante')
)
Partido_Liga_Temp = Partido_Liga_Temp.rename(columns={'Nombre_Largo_Equipo': 'Nombre_Equipo_Visitante'}).drop(columns=['ID_Equipo'])

Partido_Liga_Temp = pd.merge(
    Partido_Liga_Temp,
    Liga[['ID_Pais_Liga', 'Nombre_Liga']],
    left_on='ID_Pais_Liga',
    right_on='ID_Pais_Liga',
    how='left'
)

columnas_a_eliminar = ['ID_Partido', 'ID_Pais_Liga', 'ID_Equipo_Local', 'ID_Equipo_Visitante']
Partido_Liga_Temp = Partido_Liga_Temp.drop(columns=columnas_a_eliminar, errors='ignore')

renombres = {
    'Temporada': 'Temporada',  # Si ya se llama así, no cambia
    'Nombre_Liga': 'Liga',
    'Fecha_Partido': 'Fecha',
    'Nombre_Equipo_Local': 'Local',
    'Nombre_Equipo_Visitante': 'Visitante',
    'Goles_Local': 'Goles_Local',
    'Goles_Visitante': 'Goles_Visitante',
    'home_player_1' : 'Jugador_Local_1',
    'home_player_2' : 'Jugador_Local_2',
    'home_player_3' : 'Jugador_Local_3',
    'home_player_4' : 'Jugador_Local_4',
    'home_player_5' : 'Jugador_Local_5',
    'home_player_6' : 'Jugador_Local_6',
    'home_player_7' : 'Jugador_Local_7',
    'home_player_8' : 'Jugador_Local_8',
    'home_player_9' : 'Jugador_Local_9',
    'home_player_10' : 'Jugador_Local_10',
    'home_player_11' : 'Jugador_Local_11',
    'away_player_1' : 'Jugador_Visitante_1',
    'away_player_2' : 'Jugador_Visitante_2',
    'away_player_3' : 'Jugador_Visitante_3',
    'away_player_4' : 'Jugador_Visitante_4',
    'away_player_5' : 'Jugador_Visitante_5',
    'away_player_6' : 'Jugador_Visitante_6',
    'away_player_7' : 'Jugador_Visitante_7',
    'away_player_8' : 'Jugador_Visitante_8',
    'away_player_9' : 'Jugador_Visitante_9',
    'away_player_10' : 'Jugador_Visitante_10',
    'away_player_11' : 'Jugador_Visitante_11'
}

Partido_Liga_Temp = Partido_Liga_Temp.rename(columns=renombres)

columnas_importantes = ['Temporada', 'Liga', 'Fecha', 'Local', 'Visitante', 'Goles_Local', 'Goles_Visitante', 'Jugador_Local_1', 'Jugador_Local_2', 'Jugador_Local_3', 'Jugador_Local_4','Jugador_Local_5', 'Jugador_Local_6', 'Jugador_Local_7', 'Jugador_Local_8', 'Jugador_Local_9', 'Jugador_Local_10', 'Jugador_Local_11', 
                        'Jugador_Visitante_1', 'Jugador_Visitante_2', 'Jugador_Visitante_3', 'Jugador_Visitante_4', 'Jugador_Visitante_5', 'Jugador_Visitante_6', 'Jugador_Visitante_7', 'Jugador_Visitante_8', 'Jugador_Visitante_9', 'Jugador_Visitante_10', 'Jugador_Visitante_11']
columnas_restantes = [col for col in Partido_Liga_Temp.columns if col not in columnas_importantes]
orden_final = columnas_importantes + columnas_restantes

Partido_Liga_Temp = Partido_Liga_Temp[orden_final]


In [7]:
Jugadores1 = [f'Jugador_Local_{i}' for i in range(1, 12)] + [f'Jugador_Visitante_{i}' for i in range(1, 12)]

id_vars = [col for col in Partido_Liga_Temp.columns if col not in Jugadores1]
Partido_Liga_Temp = pd.melt(
    Partido_Liga_Temp,
    id_vars=id_vars,
    value_vars=Jugadores1,
    var_name='posicion_jugador',
    value_name='ID_Jugador'
)

partido_melted = pd.merge(
    Partido_Liga_Temp,
    Jugadores[['ID_Jugador', 'Nombre_Jugador']],
    left_on='ID_Jugador',
    right_on='ID_Jugador',
    how='left'
)

Partido_Liga_Temp = partido_melted.pivot_table(
    index=id_vars,
    columns='posicion_jugador',
    values='Nombre_Jugador',
    aggfunc='first'
).reset_index()


In [8]:
Jugadores_STATS = pd.merge(
    Jugadores[['ID_Jugador', 'Nombre_Jugador', 'Fecha_Nacimiento', 'Altura', 'Peso']],
    Atributos_Jugadores[['ID_Jugador', 'Fecha_Temporada', 'Calificación_General', 'Potencial', 'Pie_Preferido', "Aceleración", 'Velocidad_Sprint', 'Regate', 'Agilidad', 'Control_Balón',
                         'Tiros_Largos', 'Potencia_Tiro', 'Penaltis', 'Voleas', 'Portero_Salto', 'Portero_Manejo', 'Portero_Patada', 'Portero_Posicionamiento', 'Portero_Reflejos']],
    on='ID_Jugador',
    how='left'
)

Jugadores_STATS['Fecha_Nacimiento'] = pd.to_datetime(Jugadores_STATS['Fecha_Nacimiento'], errors='coerce')
Jugadores_STATS['Fecha_Temporada'] = pd.to_datetime(Jugadores_STATS['Fecha_Temporada'], errors='coerce')


Jugadores_STATS['Fecha_Nacimiento'] = pd.to_datetime(Jugadores_STATS['Fecha_Nacimiento'], errors='coerce')

Jugadores_STATS['Fecha_Temporada'] = pd.to_datetime(Jugadores_STATS['Fecha_Temporada'], errors='coerce')

Jugadores_STATS['Edad'] = Jugadores_STATS['Fecha_Temporada'].dt.year - Jugadores_STATS['Fecha_Nacimiento'].dt.year
def calcular_edad(fecha_nacimiento, fecha_temporada):
    if pd.isna(fecha_nacimiento) or pd.isna(fecha_temporada):
        return None  # Manejar fechas NaT
    nacimiento = fecha_nacimiento
    temporada = fecha_temporada
    edad = temporada.year - nacimiento.year
    if (temporada.month, temporada.day) < (nacimiento.month, nacimiento.day):
        edad -= 1
    return edad
# Aplicar la función para calcular 'Edad' correctamente
Jugadores_STATS['Edad'] = Jugadores_STATS.apply(
    lambda row: calcular_edad(row['Fecha_Nacimiento'], row['Fecha_Temporada']),
    axis=1
)

#Jugadores_STATS['Fecha_Temporada'] = pd.to_datetime(Jugadores_STATS['Fecha_Temporada'], errors='coerce')
#Jugadores_STATS['Fecha_Temporada'] = Jugadores_STATS['Fecha_Temporada'].dt.strftime('%d-%m-%y')

Jugadores_STATS = Jugadores_STATS.sort_values(['ID_Jugador', 'Fecha_Temporada']).drop_duplicates('ID_Jugador', keep='last')

Jugadores_STATS = Jugadores_STATS.sort_values(['ID_Jugador', 'Fecha_Nacimiento']).drop_duplicates('ID_Jugador', keep='last')

Jugadores_STATS = Jugadores_STATS.drop(columns=['Fecha_Nacimiento'], errors='ignore')

Orden = ['ID_Jugador', 'Nombre_Jugador', 'Edad', 'Altura', 'Peso', 'Fecha_Temporada', 'Calificación_General', 'Potencial', 'Pie_Preferido', "Aceleración", 'Velocidad_Sprint', 'Regate', 'Agilidad', 'Control_Balón',
         'Tiros_Largos', 'Potencia_Tiro', 'Penaltis', 'Voleas', 'Portero_Salto', 'Portero_Manejo', 'Portero_Patada', 'Portero_Posicionamiento', 'Portero_Reflejos']

Jugadores_STATS = Jugadores_STATS[Orden]


C:\Users\Maria Zafira\AppData\Local\Temp\ipykernel_7180\2381860454.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Jugadores_STATS['Fecha_Temporada'] = pd.to_datetime(Jugadores_STATS['Fecha_Temporada'], errors='coerce')


In [22]:
Jugadores_STATS_copia1 = Jugadores_STATS.copy()

Jugadores_STATS_copia1['Año'] = Jugadores_STATS_copia1['Fecha_Temporada'].dt.year

aceleracion_array = Jugadores_STATS_copia1['Aceleración'].to_numpy()
sprint_array = Jugadores_STATS_copia1['Velocidad_Sprint'].to_numpy()

aceleracion_promedio = np.mean([aceleracion_array, sprint_array], axis=0)

Jugadores_STATS_copia1['Rapidez'] = aceleracion_promedio.round(1)

top_10_rapidos = Jugadores_STATS_copia1.sort_values(by='Rapidez', ascending=False)

top_10_rapidos = top_10_rapidos[['Año', 'Nombre_Jugador', 'Aceleración', 'Velocidad_Sprint', 'Rapidez']].head(10)


In [23]:
Jugadores_STATS_copia2 = Jugadores_STATS.copy()

Jugadores_STATS_copia2['Año'] = Jugadores_STATS_copia2['Fecha_Temporada'].dt.year

regate_array = Jugadores_STATS_copia2['Regate'].to_numpy()
agilidad_array = Jugadores_STATS_copia2['Agilidad'].to_numpy()
control_array = Jugadores_STATS_copia2['Control_Balón'].to_numpy()

habilidades_promedio = np.mean([regate_array, agilidad_array, control_array], axis=0)

Jugadores_STATS_copia2['Habilidades_Pelota'] = habilidades_promedio.round(1)

TOP_10_Pelota = Jugadores_STATS_copia2.sort_values(by='Habilidades_Pelota', ascending=False).head(10)

TOP_10_Pelota = TOP_10_Pelota[['Año', 'Nombre_Jugador', 'Regate', 'Agilidad', 'Control_Balón', 'Habilidades_Pelota']]

In [24]:
Jugadores_STATS_copia3 = Jugadores_STATS.copy()

Jugadores_STATS_copia3['Año'] = Jugadores_STATS_copia3['Fecha_Temporada'].dt.year


tiros_largos_array = Jugadores_STATS_copia3['Tiros_Largos'].to_numpy()
potencia_tiro_array = Jugadores_STATS_copia3['Potencia_Tiro'].to_numpy()
penaltis_array = Jugadores_STATS_copia3['Penaltis'].to_numpy()
voleas_array = Jugadores_STATS_copia3['Voleas'].to_numpy()

habilidades_tiro_promedio = np.mean([tiros_largos_array, potencia_tiro_array, penaltis_array, voleas_array], axis=0)

Jugadores_STATS_copia3['Habilidades_Tiro'] = habilidades_tiro_promedio.round(1)

TOP_10_Tiro = Jugadores_STATS_copia3.sort_values(by='Habilidades_Tiro', ascending=False).head(10)

TOP_10_Tiro = TOP_10_Tiro[['Nombre_Jugador', 'Año', 'Tiros_Largos', 'Potencia_Tiro', 'Penaltis', 'Voleas', 'Habilidades_Tiro']]


In [36]:
Jugadores_STATS_copia4 = Jugadores_STATS.copy()

Jugadores_STATS_copia4['Año'] = Jugadores_STATS_copia4['Fecha_Temporada'].dt.year

salto_array = Jugadores_STATS_copia4['Portero_Salto'].to_numpy()
manejo_array = Jugadores_STATS_copia4['Portero_Manejo'].to_numpy()
patada_array = Jugadores_STATS_copia4['Portero_Patada'].to_numpy()
posicionamiento_array = Jugadores_STATS_copia4['Portero_Posicionamiento'].to_numpy()
reflejos_array = Jugadores_STATS_copia4['Portero_Reflejos'].to_numpy()

habilidades_portero_promedio = np.mean([salto_array, manejo_array, patada_array, posicionamiento_array, reflejos_array], axis=0)

Jugadores_STATS_copia4['Habilidades_Portero'] = habilidades_portero_promedio

porteros_2009 = Jugadores_STATS_copia4[Jugadores_STATS_copia4['Año'] == 2009]

top_10_porteros_2009 = porteros_2009.sort_values(by='Habilidades_Portero', ascending=False).head(10)

top_10_porteros_2009 = top_10_porteros_2009[['Nombre_Jugador', 'Portero_Salto', 'Portero_Manejo', 'Portero_Patada', 'Portero_Posicionamiento', 'Portero_Reflejos', 'Habilidades_Portero']]

top_10_porteros_2009


,Nombre_Jugador,Portero_Salto,Portero_Manejo,Portero_Patada,Portero_Posicionamiento,Portero_Reflejos,Habilidades_Portero
4839,Alberto Fontana,76.0,79.0,75.0,77.0,77.0,76.8
183535,Zeljko Kalac,75.0,73.0,71.0,73.0,75.0,73.4
69852,Henk Timmer,70.0,74.0,76.0,66.0,74.0,72.0
49415,Emanuele Manitta,76.0,68.0,66.0,76.0,72.0,71.6
44779,Dimitrios Eleftheropoulos,76.0,66.0,65.0,71.0,74.0,70.4
131001,Nelson,69.0,71.0,72.0,73.0,65.0,70.0
68728,Hans Vonk,68.0,69.0,64.0,73.0,70.0,68.8
142016,Pedro Roma,70.0,69.0,66.0,66.0,71.0,68.4
161433,Sergio Sanchez,68.0,71.0,68.0,67.0,68.0,68.4
66993,Gregorz Szamotulski,64.0,70.0,74.0,64.0,64.0,67.2


In [37]:
Jugadores_STATS_copia5 = Jugadores_STATS.copy()

Jugadores_STATS_copia5['Año'] = Jugadores_STATS_copia5['Fecha_Temporada'].dt.year

salto_array = Jugadores_STATS_copia5['Portero_Salto'].to_numpy()
manejo_array = Jugadores_STATS_copia5['Portero_Manejo'].to_numpy()
patada_array = Jugadores_STATS_copia5['Portero_Patada'].to_numpy()
posicionamiento_array = Jugadores_STATS_copia5['Portero_Posicionamiento'].to_numpy()
reflejos_array = Jugadores_STATS_copia5['Portero_Reflejos'].to_numpy()

habilidades_portero_promedio = np.mean([salto_array, manejo_array, patada_array, posicionamiento_array, reflejos_array], axis=0)

Jugadores_STATS_copia5['Habilidades_Portero'] = habilidades_portero_promedio

porteros_2010 = Jugadores_STATS_copia5[Jugadores_STATS_copia5['Año'] == 2010]

top_10_porteros_2010 = porteros_2010.sort_values(by='Habilidades_Portero', ascending=False).head(10)

top_10_porteros_2010 = top_10_porteros_2010[['Nombre_Jugador', 'Portero_Salto', 'Portero_Manejo', 'Portero_Patada', 'Portero_Posicionamiento', 'Portero_Reflejos', 'Habilidades_Portero']]

top_10_porteros_2010

,Nombre_Jugador,Portero_Salto,Portero_Manejo,Portero_Patada,Portero_Posicionamiento,Portero_Reflejos,Habilidades_Portero
150353,"Robert Enke,30",82.0,86.0,79.0,88.0,86.0,84.2
80259,Jens Lehmann,77.0,82.0,75.0,88.0,76.0,79.6
58150,Francesco Toldo,78.0,77.0,75.0,77.0,75.0,76.4
150788,Roberto Abbondanzieri,61.0,76.0,83.0,83.0,68.0,74.2
178243,Vladimir Stojkovic,78.0,71.0,70.0,72.0,80.0,74.2
15414,Antonio Chimenti,68.0,69.0,70.0,75.0,70.0,70.4
19546,Bas Roorda,64.0,71.0,67.0,69.0,66.0,67.4
30166,Christophe Revault,66.0,62.0,65.0,66.0,72.0,66.2
154912,Rui Nereu,66.0,67.0,69.0,61.0,68.0,66.2
135273,Ohad Levita,67.0,65.0,63.0,67.0,68.0,66.0


In [38]:
Jugadores_STATS_copia6 = Jugadores_STATS.copy()

Jugadores_STATS_copia6['Año'] = Jugadores_STATS_copia6['Fecha_Temporada'].dt.year

salto_array = Jugadores_STATS_copia6['Portero_Salto'].to_numpy()
manejo_array = Jugadores_STATS_copia6['Portero_Manejo'].to_numpy()
patada_array = Jugadores_STATS_copia6['Portero_Patada'].to_numpy()
posicionamiento_array = Jugadores_STATS_copia6['Portero_Posicionamiento'].to_numpy()
reflejos_array = Jugadores_STATS_copia6['Portero_Reflejos'].to_numpy()

habilidades_portero_promedio = np.mean([salto_array, manejo_array, patada_array, posicionamiento_array, reflejos_array], axis=0)

Jugadores_STATS_copia6['Habilidades_Portero'] = habilidades_portero_promedio

porteros_2011 = Jugadores_STATS_copia6[Jugadores_STATS_copia6['Año'] == 2011]

top_10_porteros_2011 = porteros_2011.sort_values(by='Habilidades_Portero', ascending=False).head(10)

top_10_porteros_2011 = top_10_porteros_2011[['Nombre_Jugador', 'Portero_Salto', 'Portero_Manejo', 'Portero_Patada', 'Portero_Posicionamiento', 'Portero_Reflejos', 'Habilidades_Portero']]

top_10_porteros_2011

,Nombre_Jugador,Portero_Salto,Portero_Manejo,Portero_Patada,Portero_Posicionamiento,Portero_Reflejos,Habilidades_Portero
48270,Edwin van der Sar,82.0,87.0,85.0,93.0,75.0,84.4
66752,Gregory Coupet,75.0,77.0,73.0,78.0,73.0,75.2
16057,Apoula Edima Edel,81.0,69.0,71.0,74.0,80.0,75.0
59304,Frank Rost,72.0,74.0,70.0,80.0,75.0,74.2
102451,Lionel Letizi,74.0,69.0,73.0,72.0,72.0,72.0
62272,Gennaro Iezzo,71.0,74.0,70.0,74.0,70.0,71.8
88905,Jose Moreira,73.0,75.0,67.0,73.0,71.0,71.8
119536,Matteo Sereni,74.0,74.0,60.0,74.0,76.0,71.6
149288,Richard Kingson,80.0,69.0,63.0,67.0,78.0,71.4
66967,Gregory Wimbee,70.0,73.0,70.0,75.0,67.0,71.0


In [40]:
Jugadores_STATS_copia7 = Jugadores_STATS.copy()

Jugadores_STATS_copia7['Año'] = Jugadores_STATS_copia7['Fecha_Temporada'].dt.year

salto_array = Jugadores_STATS_copia7['Portero_Salto'].to_numpy()
manejo_array = Jugadores_STATS_copia7['Portero_Manejo'].to_numpy()
patada_array = Jugadores_STATS_copia7['Portero_Patada'].to_numpy()
posicionamiento_array = Jugadores_STATS_copia7['Portero_Posicionamiento'].to_numpy()
reflejos_array = Jugadores_STATS_copia7['Portero_Reflejos'].to_numpy()

habilidades_portero_promedio = np.mean([salto_array, manejo_array, patada_array, posicionamiento_array, reflejos_array], axis=0)

Jugadores_STATS_copia7['Habilidades_Portero'] = habilidades_portero_promedio

porteros_2012 = Jugadores_STATS_copia7[Jugadores_STATS_copia7['Año'] == 2012]

top_10_porteros_2012 = porteros_2012.sort_values(by='Habilidades_Portero', ascending=False).head(10)

top_10_porteros_2012 = top_10_porteros_2012[['Nombre_Jugador', 'Portero_Salto', 'Portero_Manejo', 'Portero_Patada', 'Portero_Posicionamiento', 'Portero_Reflejos', 'Habilidades_Portero']]

top_10_porteros_2012

,Nombre_Jugador,Portero_Salto,Portero_Manejo,Portero_Patada,Portero_Posicionamiento,Portero_Reflejos,Habilidades_Portero
169263,Teddy Richert,77.0,73.0,74.0,78.0,79.0,76.2
68730,Hans-Joerg Butt,73.0,76.0,77.0,81.0,73.0,76.0
27240,Cesar Sanchez,75.0,73.0,72.0,85.0,71.0,75.2
46065,Doni,74.0,73.0,65.0,75.0,79.0,73.2
57691,Francesco Antonioli,72.0,74.0,60.0,80.0,76.0,72.4
171458,Tiago,70.0,74.0,71.0,74.0,71.0,72.0
118382,Mathieu Valverde,73.0,70.0,72.0,70.0,75.0,72.0
142850,Peterson Pecanha,74.0,68.0,67.0,72.0,76.0,71.4
169962,Thierry Debes,70.0,67.0,68.0,71.0,77.0,70.6
176368,Vedran Runje,74.0,70.0,66.0,68.0,75.0,70.6


In [41]:
Jugadores_STATS_copia8 = Jugadores_STATS.copy()

Jugadores_STATS_copia8['Año'] = Jugadores_STATS_copia8['Fecha_Temporada'].dt.year

salto_array = Jugadores_STATS_copia8['Portero_Salto'].to_numpy()
manejo_array = Jugadores_STATS_copia8['Portero_Manejo'].to_numpy()
patada_array = Jugadores_STATS_copia8['Portero_Patada'].to_numpy()
posicionamiento_array = Jugadores_STATS_copia8['Portero_Posicionamiento'].to_numpy()
reflejos_array = Jugadores_STATS_copia8['Portero_Reflejos'].to_numpy()

habilidades_portero_promedio = np.mean([salto_array, manejo_array, patada_array, posicionamiento_array, reflejos_array], axis=0)

Jugadores_STATS_copia8['Habilidades_Portero'] = habilidades_portero_promedio

porteros_2013 = Jugadores_STATS_copia8[Jugadores_STATS_copia8['Año'] == 2013]

top_10_porteros_2013 = porteros_2013.sort_values(by='Habilidades_Portero', ascending=False).head(10)

top_10_porteros_2013 = top_10_porteros_2013[['Nombre_Jugador', 'Portero_Salto', 'Portero_Manejo', 'Portero_Patada', 'Portero_Posicionamiento', 'Portero_Reflejos', 'Habilidades_Portero']]

top_10_porteros_2013

,Nombre_Jugador,Portero_Salto,Portero_Manejo,Portero_Patada,Portero_Posicionamiento,Portero_Reflejos,Habilidades_Portero
67916,Gustavo Adolfo Munua,74.0,71.0,86.0,73.0,78.0,76.4
144719,Quim,76.0,76.0,74.0,73.0,78.0,75.4
12829,Andres Palop,75.0,72.0,77.0,81.0,67.0,74.4
88718,Jose Manuel Pinto,78.0,75.0,78.0,68.0,69.0,73.6
163055,Simon Jentzsch,76.0,75.0,65.0,75.0,73.0,72.8
108710,Manuel Almunia,74.0,71.0,75.0,67.0,73.0,72.0
80482,Jeremie Janot,75.0,73.0,62.0,74.0,75.0,71.8
8348,Alexandros Tzorvas,74.0,65.0,64.0,70.0,77.0,70.0
92092,Julio Sergio,70.0,67.0,64.0,72.0,75.0,69.6
90463,Juan Jesus Calatayud,73.0,69.0,63.0,67.0,76.0,69.6


In [42]:
Jugadores_STATS_copia8 = Jugadores_STATS.copy()

Jugadores_STATS_copia8['Año'] = Jugadores_STATS_copia8['Fecha_Temporada'].dt.year

salto_array = Jugadores_STATS_copia8['Portero_Salto'].to_numpy()
manejo_array = Jugadores_STATS_copia8['Portero_Manejo'].to_numpy()
patada_array = Jugadores_STATS_copia8['Portero_Patada'].to_numpy()
posicionamiento_array = Jugadores_STATS_copia8['Portero_Posicionamiento'].to_numpy()
reflejos_array = Jugadores_STATS_copia8['Portero_Reflejos'].to_numpy()

habilidades_portero_promedio = np.mean([salto_array, manejo_array, patada_array, posicionamiento_array, reflejos_array], axis=0)

Jugadores_STATS_copia8['Habilidades_Portero'] = habilidades_portero_promedio

porteros_2014 = Jugadores_STATS_copia8[Jugadores_STATS_copia8['Año'] == 2014]

top_10_porteros_2014 = porteros_2014.sort_values(by='Habilidades_Portero', ascending=False).head(10)

top_10_porteros_2014 = top_10_porteros_2014[['Nombre_Jugador', 'Portero_Salto', 'Portero_Manejo', 'Portero_Patada', 'Portero_Posicionamiento', 'Portero_Reflejos', 'Habilidades_Portero']]

top_10_porteros_2014

,Nombre_Jugador,Portero_Salto,Portero_Manejo,Portero_Patada,Portero_Posicionamiento,Portero_Reflejos,Habilidades_Portero
124969,Mickael Landreau,80.0,78.0,84.0,78.0,84.0,80.8
55018,Felipe,77.0,72.0,76.0,77.0,80.0,76.4
26524,Cassio,80.0,75.0,73.0,76.0,76.0,76.0
34820,Daniel Aranzubia,75.0,71.0,78.0,73.0,76.0,74.6
43349,Dida,75.0,74.0,67.0,82.0,72.0,74.0
69520,Heinz Mueller,78.0,67.0,72.0,75.0,73.0,73.0
147722,Renan,74.0,69.0,73.0,71.0,76.0,72.6
172072,Timo Hildebrand,73.0,75.0,63.0,74.0,72.0,71.4
32570,Cristian Alvarez,75.0,68.0,69.0,68.0,75.0,71.0
87232,Jordi Codina,76.0,67.0,73.0,65.0,73.0,70.8


In [43]:
Jugadores_STATS_copia9 = Jugadores_STATS.copy()

Jugadores_STATS_copia9['Año'] = Jugadores_STATS_copia9['Fecha_Temporada'].dt.year

salto_array = Jugadores_STATS_copia9['Portero_Salto'].to_numpy()
manejo_array = Jugadores_STATS_copia9['Portero_Manejo'].to_numpy()
patada_array = Jugadores_STATS_copia9['Portero_Patada'].to_numpy()
posicionamiento_array = Jugadores_STATS_copia9['Portero_Posicionamiento'].to_numpy()
reflejos_array = Jugadores_STATS_copia9['Portero_Reflejos'].to_numpy()

habilidades_portero_promedio = np.mean([salto_array, manejo_array, patada_array, posicionamiento_array, reflejos_array], axis=0)

Jugadores_STATS_copia9['Habilidades_Portero'] = habilidades_portero_promedio

porteros_2015 = Jugadores_STATS_copia9[Jugadores_STATS_copia9['Año'] == 2015]

top_10_porteros_2015 = porteros_2015.sort_values(by='Habilidades_Portero', ascending=False).head(10)

top_10_porteros_2015 = top_10_porteros_2015[['Nombre_Jugador', 'Portero_Salto', 'Portero_Manejo', 'Portero_Patada', 'Portero_Posicionamiento', 'Portero_Reflejos', 'Habilidades_Portero']]

top_10_porteros_2015

,Nombre_Jugador,Portero_Salto,Portero_Manejo,Portero_Patada,Portero_Posicionamiento,Portero_Reflejos,Habilidades_Portero
31040,Claudio Bravo,83.0,82.0,87.0,78.0,86.0,83.2
21197,Bernd Leno,84.0,83.0,81.0,83.0,84.0,83.0
169903,Thibaut Courtois,84.0,87.0,69.0,86.0,88.0,82.8
181146,Yann Sommer,84.0,82.0,83.0,76.0,86.0,82.2
20867,Benoit Costil,82.0,80.0,78.0,82.0,85.0,81.4
177385,Vincent Enyeama,78.0,80.0,77.0,78.0,90.0,80.6
156459,Salvatore Sirigu,84.0,79.0,75.0,80.0,84.0,80.4
17755,Asmir Begovic,83.0,80.0,74.0,80.0,84.0,80.2
21363,Beto,82.0,78.0,77.0,77.0,86.0,80.0
26755,Cedric Carrasso,78.0,79.0,81.0,79.0,82.0,79.8


In [44]:
Jugadores_STATS_copia10 = Jugadores_STATS.copy()

Jugadores_STATS_copia10['Año'] = Jugadores_STATS_copia10['Fecha_Temporada'].dt.year

salto_array = Jugadores_STATS_copia10['Portero_Salto'].to_numpy()
manejo_array = Jugadores_STATS_copia10['Portero_Manejo'].to_numpy()
patada_array = Jugadores_STATS_copia10['Portero_Patada'].to_numpy()
posicionamiento_array = Jugadores_STATS_copia10['Portero_Posicionamiento'].to_numpy()
reflejos_array = Jugadores_STATS_copia10['Portero_Reflejos'].to_numpy()

habilidades_portero_promedio = np.mean([salto_array, manejo_array, patada_array, posicionamiento_array, reflejos_array], axis=0)

Jugadores_STATS_copia10['Habilidades_Portero'] = habilidades_portero_promedio

porteros_2016 = Jugadores_STATS_copia10[Jugadores_STATS_copia10['Año'] == 2016]

top_10_porteros_2016 = porteros_2016.sort_values(by='Habilidades_Portero', ascending=False).head(10)

top_10_porteros_2016 = top_10_porteros_2016[['Nombre_Jugador', 'Portero_Salto', 'Portero_Manejo', 'Portero_Patada', 'Portero_Posicionamiento', 'Portero_Reflejos', 'Habilidades_Portero']]

top_10_porteros_2016

,Nombre_Jugador,Portero_Salto,Portero_Manejo,Portero_Patada,Portero_Posicionamiento,Portero_Reflejos,Habilidades_Portero
109033,Manuel Neuer,85.0,87.0,91.0,90.0,87.0,88.0
39202,David De Gea,88.0,81.0,86.0,83.0,89.0,85.4
142868,Petr Cech,83.0,84.0,77.0,85.0,85.0,82.8
83354,Joe Hart,85.0,81.0,76.0,84.0,88.0,82.8
166130,Stephane Ruffier,83.0,81.0,77.0,83.0,86.0,82.0
109899,Marc-Andre ter Stegen,84.0,80.0,82.0,78.0,85.0,81.8
142241,Pepe Reina,80.0,76.0,82.0,85.0,84.0,81.4
63845,Gianluigi Buffon,85.0,79.0,71.0,89.0,83.0,81.4
166768,Steve Mandanda,85.0,78.0,79.0,81.0,84.0,81.4
71047,Hugo Lloris,87.0,82.0,68.0,81.0,88.0,81.2


In [ ]:
Jugadores_STATS_copia11 = Jugadores_STATS.copy()

Jugadores_STATS_copia11['Año'] = Jugadores_STATS_copia11['Fecha_Temporada'].dt.year

top_50_overall = Jugadores_STATS_copia11.sort_values(by='Calificación_General', ascending=False).head(50)

top_50_overall = top_50_overall[['Año', 'Nombre_Jugador', 'Calificación_General', 'Potencial']]

top_50_overall


,Nombre_Jugador,Año,Calificación_General,Potencial
102482,Lionel Messi,2015,94.0,94.0
33330,Cristiano Ronaldo,2015,93.0,93.0
105983,Luis Suarez,2015,90.0,90.0
109033,Manuel Neuer,2016,90.0,90.0
131464,Neymar,2016,90.0,94.0
183672,Zlatan Ibrahimovic,2015,89.0,89.0
16460,Arjen Robben,2016,89.0,89.0
122441,Mesut Oezil,2016,88.0,89.0
169805,Thiago Silva,2015,88.0,88.0
47247,Eden Hazard,2016,88.0,90.0


In [47]:
goles_local = Partido_Liga_Temp.groupby(['Local', 'Temporada'])['Goles_Local'].sum().reset_index()
goles_local = goles_local.rename(columns={'Local': 'Equipo', 'Goles_Local': 'Goles'})

goles_visitante = Partido_Liga_Temp.groupby(['Visitante', 'Temporada'])['Goles_Visitante'].sum().reset_index()
goles_visitante = goles_visitante.rename(columns={'Visitante': 'Equipo', 'Goles_Visitante': 'Goles'})

# Combinar goles local y visitante
goles_totales = pd.concat([goles_local, goles_visitante]).groupby(['Equipo', 'Temporada'])['Goles'].sum().reset_index()

# Para cada temporada, obtener el top 1 equipo con más goles
top_equipos_por_temporada = goles_totales.sort_values(['Temporada', 'Goles'], ascending=[True, False]).groupby('Temporada').head(1)

# Ordenar el resultado final por temporada y goles descendentes
top_equipos_por_temporada = top_equipos_por_temporada.sort_values(['Temporada', 'Goles'], ascending=[True, False])

# Mostrar el DataFrame directamente
top_equipos_por_temporada


,Equipo,Temporada,Goles
338,FC Barcelona,2008/2009,77
247,Chelsea,2009/2010,103
989,Real Madrid CF,2010/2011,97
990,Real Madrid CF,2011/2012,121
342,FC Barcelona,2012/2013,112
756,Manchester City,2013/2014,102
993,Real Madrid CF,2014/2015,111
994,Real Madrid CF,2015/2016,110
